In [ ]:
from pydub import AudioSegment
import os

orig_folder = "Голоса птиц"
mod_folder = "Голоса птиц мод"
f_format = "mp3"
mp3_files = [f for f in os.listdir(orig_folder) if f.lower().endswith(".mp3")]

for mp3_file in mp3_files:

    f = AudioSegment.from_file(
        os.path.join(orig_folder, mp3_file),
        f_format
    )

    # оставляем часть файла с 3 секунды без голоса диктора
    f = f[3000:]

    f.export(
        os.path.join(mod_folder, mp3_file),
        f_format
    )

print("successful end")

successful end
